# Imports

In [ ]:
# %pip install -q google-generativeai==0.8.5 vertexai==1.71.1 google-cloud-aiplatform==1.71.1 fpdf==1.7.2

In [46]:
import google.generativeai as genai
import json
from fpdf import FPDF
import re
import pandas as pd
import random
from datetime import datetime, timedelta
from io import StringIO

with open("config.json", "r") as file:
    config = json.load(file)

# Testing Gemini

In [2]:
api_key = config['api_key']
genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-2.0-flash-001")
chat = model.start_chat()
response = chat.send_message("Hello")
print("Gemini:", response.text)

Gemini: Hello! How can I help you today?



# Interior Maintenance Manual & Forms

In [53]:
manual_prompt = """
You are a technical documentation expert for a major airline. 
Generate a detailed and realistic interior inspection manual for commercial aircraft which also contains Form ID for each below section to that need to be filled in case of damage to the particular property. 
The manual should contain sections like:

1. Seat Inspection Guidelines
2. Overhead Bin and Storage Check
3. Cabin Cleanliness and Hygiene
4. Emergency Equipment Accessibility
5. In-Flight Entertainment Systems
6. Lighting and Ambient Conditions
7. Lavatory Maintenance

Each section should have bullet points or procedures with numbered steps, referencing standard airline practices. Make it suitable for internal airline staff training and inspections.

Output format:
# Aircraft Interior Inspection Manual
## Section 1: Seat Inspection Guidelines
...
"""

## Manuals

In [54]:
response = chat.send_message(manual_prompt)
print("Manual:\n", response.text[:500])

Manual:
 # Aircraft Interior Inspection Manual

**Document Version:** 3.0
**Date Issued:** October 27, 2023
**Applicability:** All Aircraft Types (Specific Aircraft Type Variants May Have Additional Requirements – Refer to Aircraft Maintenance Manual (AMM) Chapter 25)

**Purpose:** This manual provides guidelines for conducting routine interior inspections of commercial aircraft to ensure passenger safety, comfort, and adherence to airline standards. Inspectors should use this manual in conjunction with 


In [55]:
manual_text_clean = response.text
manual_text_clean = re.sub(r'[–—]', '-', manual_text_clean)  
manual_text_clean = re.sub(r'[“”]', '"', manual_text_clean)  
manual_text_clean = re.sub(r"[‘’]", "'", manual_text_clean)  
manual_text_clean = manual_text_clean.encode("ascii", "ignore").decode()  

In [56]:
class EnhancedPDF(FPDF):
    def header(self):
        self.set_font("Arial", "B", 16)
        self.cell(0, 10, "Aircraft Interior Inspection Manual", ln=True, align="C")
        self.ln(5)

    def chapter_body(self, body):
        self.set_font("Arial", "", 11)
        lines = body.split('\n')
        for line in lines:
            if line.startswith("# "):
                self.set_font("Arial", "B", 14)
                self.multi_cell(0, 10, line[2:])
                self.set_font("Arial", "", 11)

            elif line.startswith("## "):
                self.set_font("Arial", "B", 13)
                self.multi_cell(0, 10, line[3:])
                self.set_font("Arial", "", 11)
        
            else:
                segments = re.split(r"(\*\*.*?\*\*)", line)
                for segment in segments:
                    if segment.startswith("**") and segment.endswith("**"):
                        self.set_font("Arial", "B", 11)
                        self.write(10, segment[2:-2])
                        self.set_font("Arial", "", 11)
                    else:
                        self.write(10, segment)
                self.ln(10)


pdf = EnhancedPDF()
pdf.add_page()
pdf.chapter_body(manual_text_clean)

pdf_path = "aircraft_interior_inspection_manual.pdf"
pdf.output(pdf_path)

''

## Forms

In [57]:
form_prompt = f"""
You are a compliance officer creating issue reporting forms for aircraft interior inspections.
Create 5 sample filled-out synthetic reports for each section present in the given manual. At least 3 of each section reports should be those which are something visually inspectable. Each report should include:

- Form ID (taken from the manual below)
- Inspector Name
- Inspector ID
- Date
- Aircraft ID
- Inspection Zone (either Cabin or Lavatory)
- Issue Type (e.g., seat damage, dirty tray, inoperative IFE)
- Issue Description
- Severity (Low, Medium, High)
- Action Taken
- Department Contacted
- Status (Open, Resolved, Escalated)

The issues should sound realistic and align with typical cabin conditions. Make sure to vary the content.

Manual: {manual_text_clean}

Format it as structured JSON.
"""

In [58]:
response = chat.send_message(form_prompt)
print("Forms:\n", response.text[:500])


Forms:
 ```json
[
  {
    "Section": "Seat Inspection Guidelines",
    "Reports": [
      {
        "Form ID": "Form-SIG",
        "Inspector Name": "Isabelle Dubois",
        "Inspector ID": "ID7492",
        "Date": "2023-10-27",
        "Aircraft ID": "A320-XYZ123",
        "Inspection Zone": "Cabin",
        "Issue Type": "Upholstery Tear",
        "Issue Description": "Seat 32A has a 4-inch tear in the seat cushion upholstery, exposing foam.",
        "Severity": "Medium",
        "Action Taken": "


In [62]:
# print(response.text[8:-4])

In [61]:
try:
    forms_data = json.loads(response.text[8:-4])
    with open("synthetic_interior_forms.json", "w") as f:
        json.dump(forms_data, f, indent=4)
except json.JSONDecodeError:
    print("Response is not valid JSON. Please format output manually.")


## Sample data for Firebase

In [44]:
data_prompt = """You are an airline operations assistant.

Generate synthetic data in CSV format with 50 rows for aircraft inspection assignments. Each row should include the following columns:

- Inspector Name (e.g., John Davis, Priya Sharma)
- Aircraft ID (e.g., UA328, DL947, AA112)
- Date Assigned (past 90 days only, in YYYY-MM-DD format)
- Shift (Morning, Evening, Night)
- Inspection Zone (Cabin, Lavatory, Cargo Hold, Cockpit, Wings)

Ensure:
- Names are international and realistic.
- Aircraft IDs are varied and reflect different airlines.
- Dates are spread over the last 90 days.
- Distribute inspection zones and shifts randomly.

Output in **CSV format** with a header row and use only plain text (no code blocks)."""


In [45]:
response_csv = chat.send_message(data_prompt)
print("Forms:\n", response_csv.text[:500])


Forms:
 Inspector Name,Aircraft ID,Date Assigned,Shift,Inspection Zone
John Davis,UA328,2023-08-01,Morning,Cabin
Priya Sharma,DL947,2023-08-05,Evening,Lavatory
Kenji Tanaka,AA112,2023-08-10,Night,Cargo Hold
Aisha Khan,BA245,2023-08-15,Morning,Cockpit
Michael O'Connell,LH678,2023-08-20,Evening,Wings
Sofia Rossi,AF890,2023-08-25,Night,Cabin
David Lee,EK123,2023-08-30,Morning,Lavatory
Elena Petrova,QR456,2023-09-01,Evening,Cargo Hold
Omar Hassan,SQ789,2023-09-05,Night,Cockpit
Li Wei,CX012,2023-09-10,Mornin


In [48]:
df_from_gemini = pd.read_csv(StringIO(response_csv.text))

csv_gemini_path = "aircraft_inspection_assignments.csv"
df_from_gemini.to_csv(csv_gemini_path, index=False)